Samanalie Perera, 300486075

# Assignment 4: Programming 

In [2]:
from SimPy.Simulation import *
import random
import numpy as np
import math

### Question 1: A closed network has 3 nodes and 5 jobs circulating round it. All jobs start in node 0. Node 0 is an infinite-server with an exponential service time, mean 20.0. On leaving node 0, a job will transfer to node 1 with probability p or to node 2 with probability 1 − p. Node 1 is a FCFS single-server with exponential service time, mean 10.0. Jobs leaving node 1 transfer to node 0. Node 2 is a FCFS single-server with an exponential service time, mean 30.0. On leaving node 2 jobs transfer to node 1.


**Develop a SimPy simulation model of this network. Write your program to use any value of p
but set it to p = 0.5. Your task is to estimate: (1) the average time spent at each node (Wi for
each node i) and (2) show that your code calculates the average number of jobs in the system
(L) properly. For each of these performance measures produce both a point estimate and a 95%
confidence interval from 50 replications.**

In [5]:
#useful extras 
def tablelookup(P):
    u = random.random()
    sumP = 0.0
    for i in range(len(P)):
        sumP += P[i]
        if u < sumP:
            return i
        
def conf(L):
    lowerQuart = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
    upperQuart = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
    return (lowerQuart, upperQuart)

In [7]:
class Job(Process):
    def run(self,N):
        #return current time of simulation
        startTime = now()
        current = G.startNode
        
        for i in range(N):
            
            #sends request to the server 
            yield request, self, G.server[current]
            
            #random service time based on mu and it follows a exponentional distribution
            t = random.expovariate(G.mu[current])
            
            #puts the request on hold
            yield hold, self, t
            
            #releases the server 
            yield release, self, G.server[current]
           
            #to calculate the delay 
            delay = now() - startTime
            
            #records the delay
            G.delayMon[current].observe(delay)
            current = tablelookup(G.transition[current])
            
            #then sets the new start time 
            startTime = now()

#setting variables
class G:
    server = 'dummy'
    delayMon = 'Monitor'
    delayNodeMon = 'dummy'
    startNode = 0
    mu = [1/20, 1/10, 1/30]
    transition = 'dummy'

def model (N, p, maxtime, rvseed):
    #set the current simulation to 0
    initialize()
    random.seed(rvseed)
    
    #creating 3 resources, setting the first to have a capacity of 5 and the next 2 to capacity 1. 
    #monitoring to ensure the number of entities waiting is recorded 
    G.server = [Resource(capacity = 5, monitored = True),
                Resource(capacity = 1, monitored = True),
                Resource(capacity = 1, monitored = True)]
    
    #this is to record the delays
    G.delayMon = [Monitor(), Monitor(), Monitor()]
    
    #3x3 matrix that shows the probabilities following a Markov chain
    G.transition = [[0.0, p, 1-p],
                   [1.0, 0.0, 0.0], 
                   [0.0, 1.0, 0.0]]
    for i in range(5):
        j = Job(str(i))
        activate(j, j.run(N))
    simulate(until=maxtime)
        
    #mean delay for each server 
    meanDelay = []
    for i in range(len(G.delayMon)):
        #finds the mean delay of a server by using the mean function within G.delayMon. 
        meanDelay.append(G.delayMon[i].mean())
   #mean number of customers in each server
    meanCust = []
    for i in range(len(G.server)):
        
        #calculate the average waiting time and serving time and adds it to the the meanCUst list
        h = G.server[i].waitMon.mean() + G.server[i].actMon.mean()
        meanCust.append(h)
    return(meanDelay, meanCust)
    

allMeanDelay = [[],[],[]]
allMeanJobs = [[],[],[]]
for k in range(50):
    seed = 123*k
    result = model(N = 10000, p = 0.5, maxtime = 2000000, rvseed = seed)
    for i in range(3):
        allMeanDelay[i].append(result[0][i])
    for i in range(3):
        allMeanJobs[i].append(result[1][i])
L = []

for i in range(3):
    #calculating the mean number of jobs within a node
    L.append(np.mean(allMeanJobs[i]))
#expected number of jobs
exp_jobs = sum(L)
for i in range(3):
    #printing the estimated mean delay
    print("Estimate of W[", str(i), "]:", np.mean(allMeanDelay[i]))
    #printing the confidence intervals
    print("Estimate of W[", str(i), "]:", conf(allMeanDelay[i]))
for i in range(3):
    print("Estimate of L_Node[", str(i), "]:", np.mean(allMeanJobs[i]))
    print("Estimate of L_Node[", str(i), "]:", conf(allMeanJobs[i]))
print("The number of jobs within the network:", exp_jobs)

Estimate of W[ 0 ]: 20.00312180426494
Estimate of W[ 0 ]: (19.958652863954963, 20.047590744574915)
Estimate of W[ 1 ]: 19.577126549705493
Estimate of W[ 1 ]: (19.504708471315606, 19.64954462809538)
Estimate of W[ 2 ]: 87.06175004194114
Estimate of W[ 2 ]: (86.70728970096249, 87.41621038291979)
Estimate of L_Node[ 0 ]: 1.1969630755105585
Estimate of L_Node[ 0 ]: (1.1933157891103352, 1.2006103619107817)
Estimate of L_Node[ 1 ]: 1.171320461048236
Estimate of L_Node[ 1 ]: (1.1658429669313528, 1.176797955165119)
Estimate of L_Node[ 2 ]: 2.602340637279226
Estimate of L_Node[ 2 ]: (2.5945814327290755, 2.610099841829377)
The number of jobs within the network: 4.97062417383802


### Question 2: Consider a system which consists of a sequence of two M/M/2 queues. A customer enters the system and joins the waiting line for servers A. Once having been served by servers A, the customer then joins the waiting line for servers B. Once having been served by servers B, the customer leaves the system. Let section A be the waiting line for servers A plus being served by servers A, and section B be the waiting line for servers B plus being served by servers B

**Develop a SimPy simulation model of this system.
 We are interested in monitoring: (1) the average number of customers in each section;
(2) the average time a customer spends in each section; (3) the proportion of time both
servers in each of the sections are busy; and (4) the effective arrival rate of customers at
section B. For each performance measure (seven in all), produce a point estimate and a
95% confidence interval from 50 replications.
Test your simulation model, assuming interarrival times are exponentially distributed with rate
λ = 3, service times at server A are exponentially distributed with rate μA = 4 and service times
at server B are exponentially distributed with rate μB = 4.**

##Assignment 2

Consider a system which consists of a sequence of two M/M/2 queues. A customer enters the system and joins the waiting line for servers A. Once having been served by servers A, the customer then joins the waiting line for servers B. Once having been served by servers B, the customer leaves the system. Let section A be the waiting line for servers A plus being served by servers A, and section B be the waiting line for servers B plus being served by servers B.

test your simulation model, assuming interarrival times are exponentially distributed with rate λ = 3, service times at server A are exponentially distributed with rate μA = 4 and service times at server B are exponentially distributed with rate μB = 4

In [ ]:

#Going over lab 7 
# - Checks into the food queue
# - If queue is more than 5, she balks 
#(customers dont leave our line)
# - Joins food queue (queue A for us)
# - Gets to the front of the line, choose out of 3 meals
# - and take a random time (being served by A take a 
#rate of 4)
# - then they join the cashier line (Queue B for us)
# - paying takes a fixed time of 0.5 (we have a service 
#time of 4)
# - 60% prob of getting water; fixed time of 0.5 mins
#and has a exp mean 15.


In [39]:
def tablelookup(P):
    u = random.random()
    sumP = 0.0
    for i in range(len(P)):
        sumP += P[i]
        if u < sumP:
            return i
def conf(L):
    """confidence interval"""
    lower = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
    upper = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
    return lower, upper

class Source(Process):
    """generate random arrivals"""
    def run(self, N, lamb, muA, muB):
        for i in range(N):
            a = Arrival(str(i))
            activate(a, a.run(muA, muB))
            t = random.expovariate(lamb)
            yield hold, self, t
            
class Arrival(Process):
    n = 0
    
    def run(self, muA, muB):
        Arrival.n += 1
        arrivalA = Arrival.n
        startTime = now()
        
        #Section A
        G.numberMonA.observe(arrivalA) #total numbermon
        G.numberMonA.observe(arrivalA) # section A numbermon
        if(arrivalA > 0):
            G.busyMonA.observe(1)
        else:
            G.busyMonA.observe(0)
            
        yield request, self, G.queueA
        a = random.expovariate(muA)
        yield hold, self, a
        yield release, self, G.queueA
        
        arrivalA -= 1 #leaving queueA
        G.numberMonA.observe(arrivalA)
        if(arrivalA > 0):
            G.busyMonA.observe(1)
        else:
            G.busyMonA.observe(0)
        delay = now() - startTime
        G.delayMonA.observe(delay)
        
        #Section B
        arrivalTimeB = now()
        G.numberMonB.observe(Arrival.n)
        if(Arrival.n > 0):
            G.busyMonB.observe(1)
        else:
            G.busyMonB.observe(0)
        yield request, self, G.queueB
        b = random.expovariate(muB)
        yield hold, self, b
        yield release, self, G.queueB
        
        Arrival.n -= 1 #leaving queueA
        G.numberMonB.observe(Arrival.n)
        G.numberMonB.observe(Arrival.n)
        if(Arrival.n > 0):
            G.busyMonB.observe(1)
        else:
            G.busyMonB.observe(0)
            
        delayB = now() - startTime
        G.delayMonB.observe(delayB)
        
        totalDelayTime = now() - arrivalA
        G.delayMon.observe(totalDelayTime)
        

        
class G:
    queueA = 'dummy'
    queueB = 'dummy'
    
    delayMon = 'Monitor'
    delayMonA = 'Monitor'
    delayMonB = 'Monitor'
    
    numberMon = 'Monitor'
    numberMonA = 'Monitor'
    numberMonB = 'Monitor'
    
    busyMonA = 'Monitor'
    busyMonB = 'Monitor'

def model(c, N, lamb, muA, muB, maxtime, rvseed):
    # setup
    initialize()
    random.seed(rvseed)
    
    G.queueA = Resource(2, monitored = True)
    G.queueB = Resource(2, monitored = True)
        
    G.delayMon = Monitor()
    G.delayMonA = Monitor()
    G.delayMonB = Monitor()

    G.numberMon = Monitor()
    G.numberMonA = Monitor()
    G.numberMonB = Monitor()
    
    G.busyMonA = Monitor()
    G.busyMonB = Monitor()

        
    # simulate
    s = Source('Source')
    activate(s, s.run(N, lamb, muA, muB))
    simulate(until=maxtime)
        
    # gather performance measures
    meanCustA = []
    for customer in G.queueA.waitMon:
        # calculate the average waiting time and service time and add it to the meanCustA list
        h = G.queueA.waitMon[int(customer)] + G.queueA.actMon[int(customer)]
        meanCustA.append(h)
        
    meanCustB = []
    for customer in G.queueB.waitMon:
        # calculate the average waiting time and service time and add it to the meanCustB list
        h = G.queueB.waitMon[int(customer)] + G.queueB.actMon[int(customer)]
        meanCustB.append(h)
        
    W = G.delayMon.mean()
    L = G.numberMon.mean()
    return W, L, meanCustA, meanCustB
    
    

allMeanCustA = []
allMeanCustB = []

for k in range(50):
    seed = 123*k
    result = model(c=1, N=10000, lamb=3, muA = 4, muB = 4, maxtime=180, rvseed=seed)
    allMeanCustA.append(result[0])
    allMeanCustB.append(result[1])
    
print("=================")
print("lambda =", 3)
print("Estimate of A:", np.mean(allMeanCustA[k]))
print("Estimate of B:", np.mean(allMeanCustB[k]))
print("Estimate of A:", conf(allMeanCustA[k]))
print("Estimate of B:", conf(allMeanCustB[k]))

TypeError: int() argument must be a string, a bytes-like object or a number, not 'list'

In [38]:
#two queues
#complete queue a, then queue b.
#Probability for current state, ignore past/future.
#Time between arrival of "queue objects" is random

#Monitor the following performance measures
# - Average number of customers in each queue
# - Average time spent in a queue (per customer)
# - Percentage of time "checkout lady" is busy in their own line
# - Arrival rate for queue B. (How often a customer enters queue B)

#For each performance measure we need the following
#Confidence interval required 95% from 50 replications.
#Point estimate required.



#PSUEDO

#for loop 50 times (50 replications)

#CODE BELOW



# from SimPy.Simulation import *
# import random
# import numpy as np
# import math
# def conf(L):
#     """confidence interval"""
#     lower = np.mean(L) - 1.96*np.std(L)/math.sqrt(len(L))
#     upper = np.mean(L) + 1.96*np.std(L)/math.sqrt(len(L))
#     return lower, upper

# class Source(Process):
#     """generate random arrivals"""
#     def run(self, N, lamb):
#         for i in range(N):
#             a = Arrival(str(i))
#             activate(a, a.run())
#             t = random.expovariate(lamb)
#             yield hold, self, t
            
# class Arrival(Process):
#     """an arrival"""
#     n = 0 # class variable (number in system)
#     def run(self):
#         if len(G.food.waitQ) <= 5:
#             G.balkmon.observe(0)
#             Arrival.n += 1 # number in system
#             arrivetime = now()
#             G.numbermon.observe(Arrival.n)
#             ## food
#             yield request, self, G.food
#             for i in range(3):
#                 t = random.uniform(0.5,1)
#                 yield hold, self, t
#             yield release, self, G.food
#             ## cashier
#             yield request, self, G.cashier
#             yield hold, self, 0.5
#             yield release, self, G.cashier
#             ## cooler
#             if (random.random()<0.6):
#                 G.watermon.observe(1)
#                 yield request, self, G.cooler
#                 yield hold, self, 0.5
#                 yield release, self, G.cooler
#             else:
#                 G.watermon.observe(0)
#             ## eat
#             yield hold, self, random.expovariate(1.0/15)
#             Arrival.n -= 1
#             G.numbermon.observe(Arrival.n)
#             delay = now()-arrivetime
#             G.delaymon.observe(delay)
#         else:
#             G.balkmon.observe(1)
            
# class G:
#     food = 'dummy'
#     cashier = 'dummy'
#     cooler = 'dummy'
#     delaymon = 'Monitor'
#     numbermon = 'Monitor'
#     balkmon = 'Monitor'
#     watermon = 'Monitor'
    
# def model(c, N, lamb, maxtime, rvseed):
#     # setup
#     initialize()
#     random.seed(rvseed)
#     G.food = Resource()
#     G.cashier = Resource()
#     G.cooler = Resource()
#     G.delaymon = Monitor()
#     G.numbermon = Monitor()
#     G.balkmon = Monitor()
#     G.watermon = Monitor()
#     Arrival.n = 0
#     # simulate
#     s = Source('Source')
#     activate(s, s.run(N, lamb))
#     simulate(until=maxtime)
#     # gather performance measures
#     W = G.delaymon.mean()
#     L = G.numbermon.timeAverage()
#     Pbalk = G.balkmon.mean()
#     Pwater = G.watermon.mean()
#     return W, L, Pbalk, Pwater

# ## Experiment ----------------
# for thelambda in [0.1, 0.4, 0.8]:
#     allW = []
#     allL = []
#     allPbalk = []
#     allPwater = []
#     for k in range(50):
#         seed = 123*k
#         result = model(c=1, N=10000, lamb=thelambda, maxtime=180, rvseed=seed)
#         allW.append(result[0])
#         allL.append(result[1])
#         allPbalk.append(result[2])
#         allPwater.append(result[3])
#     print("=================")
#     print("lambda =", thelambda)
#     print(" Estimate of W:", np.mean(allW))
#     print(" Conf int of W:", conf(allW))
#     print(" Estimate of L:", np.mean(allL))
#     print(" Conf int of L:", conf(allL))
#     print(" Estimate of Pbalk:", np.mean(allPbalk))
#     print(" Conf int of Pbalk:", conf(allPbalk))
#     print(" Estimate of Pwater:", np.mean(allPwater))
#     print(" Conf int of Pwater:", conf(allPwater))